In [28]:
import numpy as np
import pandas as pd
import re
import jieba

In [29]:
data=pd.read_excel("./data/data_demo.xlsx")
data.head()

,id,pid,contxt,diag_code,tcms_code,tan,feng,ren,yinxu,fei
0,1,1803613,右肺癌术后三月余。 患者面色憔悴；形体消瘦，无咳喘，无喉中水鸡声；言语清晰、流畅；气息平和；...,0,2,0,0,0,1,1
1,2,1802607,右肺癌术后两月余。 患者面色憔悴；形体消瘦，无咳喘，无喉中水鸡声；言语清晰、流畅；气息平和；...,0,2,0,0,0,1,1
2,3,1809724,右肺癌术后8年余，咳嗽气喘加重1周。 患者气喘貌，无喉中水鸡声；言语清晰、流畅；气息尚平和；...,0,2,0,0,0,1,1
3,4,1710003,右肺癌术后8年余，反复咳嗽气喘半年余。 患者气喘貌，无喉中水鸡声；言语清晰、流畅；气息尚平和...,0,2,0,0,0,1,1
4,5,1613461,右肺癌术后7年余，咳嗽气喘20小时。 患者气喘貌，无喉中水鸡声；言语清晰、流畅；气息尚平和；...,0,2,0,0,0,1,1


In [30]:
y=data.tcms_code
tcme=data.iloc[:,5:]
context=data.contxt
diag=data.diag_code

In [31]:
print(y.shape,tcme.shape,context.shape,diag.shape)

(1400,) (1400, 5) (1400,) (1400,)


In [32]:
dictfile="./data/tcm_all_dict_2.txt"
stopwords="./data/stopwords.txt"
jieba.load_userdict(dictfile)   

In [33]:
def file2doc1(infile,dictfile):    
    #jieba.load_userdict(dictfile)                     
    doc = [w for x in codecs.open(file_name, 'r', 'utf-8').readlines() for w in jieba.cut(x.strip())]
    return doc

In [34]:
def load_stopword(filename):  
    stopwords = [line.strip() for line in open(filename, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [35]:
def drop_alnum(astr):
    drop_str="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    str1=astr
    for ch in str1:
        if ch in drop_str:
            str1=str1.replace(ch,"")
    return str1

In [36]:
def del_neg_txt1(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="无"   
    item1=["；","，","舌"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [37]:
def del_neg_txt2(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="未"
    item1=["；","，","有"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [38]:
def get_text_neg(context):
    corpus=[]    
    
    for sent in context:
        sent=sent+"；"
        sent=del_neg_txt1(sent) #drop negative words  
        sent=del_neg_txt2(sent)
        sent.strip("；")        
        corpus.append(sent)
        
    return corpus

In [39]:
txt_ng = get_text_neg(context)

In [40]:
txt_pd=pd.DataFrame(txt_ng)
txt_pd.to_excel("txt_pos.xls")

In [41]:
def get_corpus(context):
    corpus=[]    
    
    for sent in context:
        sent=sent+"；"
        sent=del_neg_txt1(sent) #drop negative words  
        sent=del_neg_txt2(sent)
        sent.strip("；")
        strlist=list(jieba.cut(sent))
        strlist=sorted(set(strlist),key=strlist.index)
        corpus.append(strlist)
        
    return corpus

In [42]:
def get_corpus1(doc,stopwords):    
    stopword_list=load_stopword(stopwords)   
    #r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'   #drop letters, numbers and sepcial chars
    out=[]
    
    for sent in doc:        
        doc_stopword=[y for y in sent if y not in stopword_list] #drop stopword
        res=[]
        for y in doc_stopword:
            #res.append(re.sub(r1,"",y))   #drop letters, numbers and sepcial chars
            y=drop_alnum(y)
            res.append(y)  #drop letters, numbers and sepcial chars
        out.append(res)
    return out

In [43]:
atxt=get_corpus(context)
cps=get_corpus1(atxt, stopwords)

In [44]:
#TextCNN - TCM syndrome

In [45]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [46]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [47]:
y=data.tcms_code
all_texts=cps
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y),num_classes=None)

Found 1008 unique tokens.


In [48]:
from sklearn.model_selection import train_test_split
x=datatxt
y1=labels
x_train, x_test, y_train, y_test=train_test_split(x,y1,test_size=0.2,random_state=None)

In [49]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
#EMBEDDING_DIM = 128

In [50]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 64, 256)           258304    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 512)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4194560   
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total para

In [51]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=32, epochs=8, verbose=1, validation_split=0.2, shuffle=True) 
#score1 = model.evaluate(x_train,y_train, verbose=0)
score2 = model.evaluate(x_test,y_test, verbose=0)
print("accuracy:",score2[1])

Train on 896 samples, validate on 224 samples
Epoch 1/8
896/896 [==============================] - 4s 4ms/step - loss: 1.0582 - acc: 0.5379 - val_loss: 0.6862 - val_acc: 0.7500
Epoch 2/8
896/896 [==============================] - 3s 4ms/step - loss: 0.3965 - acc: 0.8549 - val_loss: 0.2549 - val_acc: 0.8973
Epoch 3/8
896/896 [==============================] - 3s 4ms/step - loss: 0.1444 - acc: 0.9531 - val_loss: 0.1668 - val_acc: 0.9286
Epoch 4/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0782 - acc: 0.9766 - val_loss: 0.1711 - val_acc: 0.9196
Epoch 5/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0476 - acc: 0.9855 - val_loss: 0.1677 - val_acc: 0.9241
Epoch 6/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0300 - acc: 0.9911 - val_loss: 0.2356 - val_acc: 0.9196
Epoch 7/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0332 - acc: 0.9900 - val_loss: 0.2148 - val_acc: 0.9286
Epoch 8/8
896/896 [==================

In [15]:
#TextCNN - TCM element

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [17]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()

In [19]:
ym=pd.DataFrame(tcme)

all_texts=cps
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#labels = to_categorical(np.asarray(y),num_classes=None)
print('Shape of data tensor:', datatxt.shape)
print('Shape of label tensor:', ym.shape)
print(word_index)

Found 1008 unique tokens.
Shape of data tensor: (1400, 64)
Shape of label tensor: (1400, 5)
{' ': 1, '咳嗽': 2, '咳痰': 3, '患者': 4, '为主': 5, '舌质': 6, '精神': 7, '面色': 8, '诉': 9, '结合': 10, '红': 11, '苔': 12, '舌脉': 13, '痰': 14, '清晰': 15, '阵发性': 16, '天': 17, '发热': 18, '二便': 19, '言语': 20, '伴': 21, '脉滑': 22, '胸闷': 23, '稍': 24, '气喘': 25, '正常': 26, '神清': 27, '咯痰': 28, '形体': 29, '双目': 30, '呼吸': 31, '发育': 32, '营养': 33, '应答': 34, '自如': 35, '语声': 36, '调': 37, '少神': 38, '有神': 39, '清': 40, '黄腻': 41, '食纳': 42, '少华': 43, '舌红': 44, '平稳': 45, '适中': 46, '行动自如': 47, '淡红': 48, '白腻': 49, '加重': 50, '后': 51, '表情': 52, '声音洪亮': 53, '脉浮': 54, '月余': 55, '尚可': 56, '神志': 57, '不易': 58, '活动': 59, '苔薄': 60, '咯出': 61, '气促': 62, '一周': 63, '': 64, '精神不振': 65, '数': 66, '黄': 67, '脉滑数': 68, '中等': 69, '脉细': 70, '症': 71, '脉细数': 72, '响亮': 73, '苔少': 74, '肺癌': 75, '乏力': 76, '患儿': 77, '良好': 78, '余': 79, '咯': 80, '粘': 81, '欠佳': 82, '自然': 83, '三天': 84, '咯吐': 85, '粘痰': 86, '十余天': 87, '时有': 88, '伴有': 89, '月': 90, '痛苦': 91, '咳声': 92, '纳差': 

In [20]:
from sklearn.model_selection import train_test_split
x=datatxt
#ym=pd.DataFrame(outm)
x_train, x_test, y_train, y_test=train_test_split(x,ym,test_size=0.20,random_state=0)

In [21]:
y_test.head()

,tan,feng,ren,yinxu,fei
1110,0,1,1,0,1
310,1,0,1,0,1
776,1,0,0,0,1
491,1,0,1,0,1
1063,0,1,1,0,1


In [22]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
#EMBEDDING_DIM = 128

In [23]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
#model.add(Dense(ym.shape[1], activation='softmax')) #multiclass
model.add(Dense(ym.shape[1], activation='sigmoid')) #multilabel
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 256)           258304    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 512)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_1 (Dense)      

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"]) 
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=32, epochs=8, verbose=1, validation_split=0.2, shuffle=True)
score = model.evaluate(x_test,y_test, verbose=0)
print(score[1])

Instructions for updating:
Use tf.cast instead.
Train on 896 samples, validate on 224 samples
Epoch 1/8
896/896 [==============================] - 4s 4ms/step - loss: 0.3457 - acc: 0.8429 - val_loss: 0.2034 - val_acc: 0.9268
Epoch 2/8
896/896 [==============================] - 3s 3ms/step - loss: 0.1059 - acc: 0.9629 - val_loss: 0.1247 - val_acc: 0.9562
Epoch 3/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0480 - acc: 0.9801 - val_loss: 0.1193 - val_acc: 0.9625
Epoch 4/8
896/896 [==============================] - 4s 4ms/step - loss: 0.0248 - acc: 0.9911 - val_loss: 0.1196 - val_acc: 0.9634
Epoch 5/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0130 - acc: 0.9953 - val_loss: 0.1285 - val_acc: 0.9598
Epoch 6/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0088 - acc: 0.9964 - val_loss: 0.1320 - val_acc: 0.9625
Epoch 7/8
896/896 [==============================] - 3s 4ms/step - loss: 0.0064 - acc: 0.9975 - val_loss: 0.1408 - val_a